In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import toolz as tz

In [31]:
import pandas
pandas.set_option('display.max_rows', None)

In [331]:
from absbox import Generic

test01 = Generic(
    "TEST01"
    ,{"cutoff":"2021-03-01","closing":"2021-06-15","firstPay":"2021-07-26"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2025-01-01"}
    ,{'assets':[["Mortgage" ,{"originBalance":0,"originRate":["fix",0.045],"originTerm":30
                            ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
                    ,{"currentBalance":0 ,"currentRate":0.08 ,"remainTerm":24 ,"status":"current"}]]}
    ,(("acc01",{"balance":0}),("buyAcc",None))
    ,tuple()
    ,(("trusteeFee",{"type":{"fixFee":30}}),)
    ,{("DistributionDay","Warehousing"):[
         ["payFee","acc01",['trusteeFee']]
         ,["liqAccrue","warehouse"]
         ,["liqSupport", "warehouse", "account", "buyAcc", {"formula":("custom","buyCurve")}]
         ,["IfElse",["<","2022-08-20"]
            ,[["buyAsset",["Current|Defaulted",0.3,0],"buyAcc",None]]
            ,[
                #["liqRepay","fee","acc01","warehouse"],
                #["liqRepay","fee","acc01","warehouse"],
                ["liqRepay",["int","fee","bal"],"acc01","warehouse"],
            ]
            ]
        
     ]
    ,"cleanUp":[
        ["sellAsset", ["PvRate", 0.08], "acc01"] 
        #["sellAsset", ["Current|Defaulted", 0.9, 0.2], "acc01"] 
    ]}
    ,[["CollectedCash","acc01"]]
    ,{"warehouse":{
        "credit" : 2000
        ,"balance":300
        ,"type": {"total": 4000}
        ,"creditCalc":"IncludeDueInt"
        ,"start": "2021-06-15"
        ,"rateType": ("fix", 0.05)
        ,"feeType": ("fix", 0.03)
        ,"rate":0.03
        ,"fee":0.01
        }
     }
    ,None
    ,None
    ,None
    ,"Warehousing"
    ,{"buyCurve": {"BalanceCurve": [
        ["2022-01-01",500]
        ,["2022-06-01",1000]
    ] 
    }})

In [332]:
from absbox import API,EnginePath
localAPI = API(EnginePath.LOCAL,check=False)

Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.28.7, server:0.29.20

In [333]:
revol_asset = ["Mortgage"
                ,{"originBalance":10,"originRate":["fix",0.045],"originTerm":30
                  ,"freq":"Monthly","type":"Level","originDate":"2021-07-01"}
                  ,{"currentBalance":10
                  ,"currentRate":0.08
                  ,"remainTerm":12
                  ,"status":"current"}]


r = localAPI.run(test01,
           runAssump=[("revolving"
                         ,["constant",revol_asset]
                         ,("Pool",("Mortgage",None,None,None,None)
                                   ,None
                                   ,None))
                      ,("inspect",("MonthEnd",[("poolBalance",),("bondBalance",)]))
                      ,("call",("afterDate","2022-09-20"))
                      #,("stop","2022-06-20")
                       ],
           read=True)

Warning Message from server:
Account acc01 has cash to be distributed
Account buyAcc has cash to be distributed

In [334]:
r['accounts']['acc01'].tail()

,balance,change,memo
date,,,
2022-08-31,564.28,564.28,<Pool:CollectedCash>
2022-09-20,564.28,0.00,<SeqPayFee:trusteeFee>
2022-09-20,467.01,-97.27,<Support:warehouse>
2022-09-30,1031.29,564.28,<Pool:CollectedCash>
2022-10-20,4065.71,3034.42,<Liquidation:>


In [335]:
r['result']['status']

,Date,From,To
0,2022-10-20,Warehousing,Called
1,2022-10-20,DealEnd,Clean Up
